# Processamento de Imagem e Visão
## Trabalho Prático 1
### Alunos: Belarmino Sacate (52057) e Miguel Ferreira (51878)

------
# 1. INTRODUÇÃO

-------------
## BIBLIOTECAS

In [21]:
import cv2
import numpy as np

------------
## 3.VARRIAVEIS

In [22]:
CAMINHO_VIDEO = 'Vídeo - Enunciado A.avi'

# Parâmetros de Processamento
AREA_MINIMA_CONTORNO = 600
LIMIAR_DISTANCIA = 50
HISTORICO_FUNDO = 50

# Variáveis de Estado Global
VEICULOS_RASTREADOS = {}  
CONTADOR_ID_VEICULOS = 0   
ESTADO_ROI = {'pontos': [], 'modo_input': True}

------------
## METODOS

Estima a imagem de fundo através da mediana temporal de frames aleatórios

In [23]:
def obter_modelo_fundo(caminho_video, num_frames):
    
    captura = cv2.VideoCapture(caminho_video)
    total_frames = int(captura.get(cv2.CAP_PROP_FRAME_COUNT))
    
    indices_frames = np.random.randint(0, total_frames, num_frames)
    frames = []
    
    for indice in indices_frames:
        captura.set(cv2.CAP_PROP_POS_FRAMES, indice)
        sucesso, frame = captura.read()
        if sucesso:
            frames.append(frame)
            
    captura.release()
    
    if not frames:
        raise ValueError("Falha ao ler frames para o modelo de fundo.")

    imagem_fundo = np.median(frames, axis=0).astype(dtype=np.uint8)
    print("Fundo gerado.")
    return imagem_fundo

Callback para registar os 4 cliques do mouse na ROI

In [24]:
def callback_mouse(evento, x, y, flags, param):
    
    if evento == cv2.EVENT_LBUTTONDOWN and ESTADO_ROI['modo_input']:
        if len(ESTADO_ROI['pontos']) < 4:
            ESTADO_ROI['pontos'].append((x, y))
            print(f"Ponto registado: {x}, {y}")

Permite ao utilizador definir a ROI com 4 cliques no primeiro frame

In [25]:
def obter_roi_utilizador(caminho_video):
    
    # Reinicializa o estado
    ESTADO_ROI['pontos'] = []
    ESTADO_ROI['modo_input'] = True
    
    captura = cv2.VideoCapture(caminho_video)
    sucesso, frame = captura.read()
    captura.release()
    
    if not sucesso: return None

    nome_janela = "Selecione a Area (4 Cliques) e Pressione TECLA"
    cv2.namedWindow(nome_janela)
    cv2.setMouseCallback(nome_janela, callback_mouse)
    
    print("\n>>> Clique em 4 pontos para definir a ROI. Pressione uma tecla para continuar.")

    while True:
        quadro_visual = frame.copy()
        
        pontos = ESTADO_ROI['pontos']
        if len(pontos) > 0:
            for pt in pontos:
                cv2.circle(quadro_visual, pt, 5, (0, 0, 255), -1)
            
            if len(pontos) > 1:
                pts_np = np.array(pontos, np.int32).reshape((-1, 1, 2))
                cv2.polylines(quadro_visual, [pts_np], False, (0, 255, 0), 2)
            
            if len(pontos) == 4:
                cv2.line(quadro_visual, pontos[-1], pontos[0], (0, 255, 0), 2)

        cv2.imshow(nome_janela, quadro_visual)
        
        key = cv2.waitKey(20) & 0xFF
        if len(pontos) == 4 and key != 255:
            break
            
    cv2.destroyWindow(nome_janela)
    ESTADO_ROI['modo_input'] = False
    return np.array(pontos, dtype=np.int32)

Subtração de fundo e binarização

In [26]:
def processar_plano_frente(quadro_cinza, fundo_cinza):
    diferenca = cv2.absdiff(quadro_cinza, fundo_cinza)
    _, limiarizado = cv2.threshold(diferenca, 30, 255, cv2.THRESH_BINARY)
    return limiarizado

Aplica operadores morfológicos (fecho/dilatação) e a máscara da ROI

In [27]:
def aplicar_morfologia_e_roi(mascara_limiarizada, roi_pontos):
    
    kernel = np.ones((5,5), np.uint8)
    mascara = cv2.morphologyEx(mascara_limiarizada, cv2.MORPH_CLOSE, kernel)
    mascara = cv2.dilate(mascara, kernel, iterations=2)
    
    
    mascara_roi_img = np.zeros_like(mascara)
    cv2.fillPoly(mascara_roi_img, [roi_pontos], 255)
    mascara_processada = cv2.bitwise_and(mascara, mascara_roi_img)
    
    return mascara_processada

Detecção de regiões ativas (contornos)

In [28]:
def detetar_veiculos(mascara_processada, area_minima):
    
    contornos, _ = cv2.findContours(mascara_processada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    detecoes = []
    for contorno in contornos:
        area = cv2.contourArea(contorno)
        if area > area_minima:
            x, y, w, h = cv2.boundingRect(contorno)
            cx, cy = x + w // 2, y + h // 2
            detecoes.append((cx, cy, x, y, w, h))
    return detecoes

    Faz a correspondência das deteções com o histórico e atribui novos IDs.
    Retorna (deteções_com_id, novos_veiculos_rastreados, novo_contador_global).

In [29]:
def atualizar_rastreamento(detecoes_atuais, historico_veiculos, limiar_distancia, contador_global_id):

    novos_veiculos = {}
    deteccoes_com_id = []
    
    novo_contador_global_id = contador_global_id 
    
    for (cx, cy, x, y, w, h) in detecoes_atuais:
        id_atual = -1
        
    
        for vid, (old_cx, old_cy) in historico_veiculos.items():
            distancia = np.sqrt((cx - old_cx)**2 + (cy - old_cy)**2)
            if distancia < limiar_distancia:
                id_atual = vid
                break
        
        if id_atual != -1:
            novos_veiculos[id_atual] = (cx, cy)
        else:
            novo_contador_global_id += 1
            id_atual = novo_contador_global_id
            novos_veiculos[id_atual] = (cx, cy)
            
        deteccoes_com_id.append((cx, cy, x, y, w, h, id_atual))

    return deteccoes_com_id, novos_veiculos, novo_contador_global_id

Desenha Bounding Boxes, IDs e contagem no frame

In [30]:
def desenhar_resultados(quadro_visual, deteccoes_com_id, roi_pontos, contagem_total):
    
    for cx, cy, x, y, w, h, id_atual in deteccoes_com_id:
    
        cv2.rectangle(quadro_visual, (x, y), (x + w, y + h), (0, 0, 255), 2)
      
        cv2.putText(quadro_visual, f"V{id_atual}", (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


    cv2.polylines(quadro_visual, [roi_pontos], True, (0, 255, 0), 2)
    

    cv2.putText(quadro_visual, f"Total: {contagem_total}", (20, 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

EXECUÇÃO PRINCIPAL

In [31]:
# Calibração do Fundo
print("A calcular modelo de fundo...")
try:
    imagem_fundo = obter_modelo_fundo(CAMINHO_VIDEO, HISTORICO_FUNDO)
    fundo_cinza = cv2.cvtColor(imagem_fundo, cv2.COLOR_BGR2GRAY)
except Exception as e:
    raise RuntimeError(f"Erro: {e}")

# Calibração da ROI
roi_selecionada = obter_roi_utilizador(CAMINHO_VIDEO)
if roi_selecionada is None or len(roi_selecionada) != 4:
    raise RuntimeError("Erro: ROI não definida. A execução será interrompida.")

# Inicialização do Vídeo
captura = cv2.VideoCapture(CAMINHO_VIDEO)
print("\nProcessamento iniciado. Pressione ESC para sair na janela de vídeo.")



# --- LOOP DE PROCESSAMENTO DE VÍDEO ---

while True:
    sucesso, frame = captura.read()
    if not sucesso:
        break 
        
    quadro_visual = frame.copy()
    quadro_cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 1. Subtração de Fundo
    mascara_limiarizada = processar_plano_frente(quadro_cinza, fundo_cinza)
    
    # 2. Morfologia e ROI
    mascara_processada = aplicar_morfologia_e_roi(mascara_limiarizada, roi_selecionada)
    
    # 3. Detecção de Contornos
    detecoes_atuais = detetar_veiculos(mascara_processada, AREA_MINIMA_CONTORNO)
    
    # 4. Rastreamento (Tracking)
    # Atualiza o estado global (veiculos_rastreados e contador_id_veiculos)
    resultados_deteccao, novos_veiculos_rastreados, novo_contador_id_veiculos = \
        atualizar_rastreamento(detecoes_atuais, VEICULOS_RASTREADOS, LIMIAR_DISTANCIA, CONTADOR_ID_VEICULOS)
    
    VEICULOS_RASTREADOS = novos_veiculos_rastreados
    CONTADOR_ID_VEICULOS = novo_contador_id_veiculos

    # 5. Visualização
    desenhar_resultados(quadro_visual, resultados_deteccao, roi_selecionada, CONTADOR_ID_VEICULOS)

    cv2.imshow("Contagem de Veiculos", quadro_visual)
    
    if cv2.waitKey(30) == 27:
        break

#6finalizacao
captura.release()
cv2.destroyAllWindows()
print("Processamento finalizado.")

A calcular modelo de fundo...
Fundo gerado.

>>> Clique em 4 pontos para definir a ROI. Pressione uma tecla para continuar.
Ponto registado: 96, 84
Ponto registado: 157, 86
Ponto registado: 196, 182
Ponto registado: 35, 196

Processamento iniciado. Pressione ESC para sair na janela de vídeo.
Processamento finalizado.


------------
# 5. CONCLUSÃO